# Support Functions

## String to List

In [1]:
def stringToList(mess):
  input = []
  for i in range(4):
    start = int(i*len(mess)/4)
    end = int((i+1)*len(mess)/4)
    input.append(mess[start:end])
  return input

## Convert Stream to Matrix and visa versa

In [2]:
def TransformData(mess):
  res = []
  for i in range(len(mess)):
    temp_res = []
    for j in range(len(mess[0])):
      temp_res.append(mess[j][i])
    res.append(temp_res)
  return res

## Print Out Message

In [3]:
def printMessage(val):
  str = ""
  for i in val:
    for j in i:
      str+=j+" "
  return str

## RoundKey Addition(ARK)

In [4]:
def RoundKeyAddition(input,key):
  ARK = [[]for i in range(4)]
  for i in range(4):
    for j in range(4):
      ARK[i].append(format((int(input[i][j],16) ^ int(key[i][j],16)),'#04x')[2:])
  return ARK

## S-Box

In [5]:
s_box = ['63	7c	77	7b	f2	6b	6f	c5	30	01	67	2b	fe	d7	ab	76'.split("\t"),'ca	82	c9	7d	fa	59	47	f0	ad	d4	a2	af	9c	a4	72	c0'.split("\t"),'b7	fd	93	26	36	3f	f7	cc	34	a5	e5	f1	71	d8	31	15'.split("\t"),'04	c7	23	c3	18	96	05	9a	07	12	80	e2	eb	27	b2	75'.split("\t"),'09	83	2c	1a	1b	6e	5a	a0	52	3b	d6	b3	29	e3	2f	84'.split("\t"),'53	d1	00	ed	20	fc	b1	5b	6a	cb	be	39	4a	4c	58	cf'.split("\t"),'d0	ef	aa	fb	43	4d	33	85	45	f9	02	7f	50	3c	9f	a8'.split("\t"),'51	a3	40	8f	92	9d	38	f5	bc	b6	da	21	10	ff	f3	d2'.split("\t"),'cd	0c	13	ec	5f	97	44	17	c4	a7	7e	3d	64	5d	19	73'.split("\t"),'60	81	4f	dc	22	2a	90	88	46	ee	b8	14	de	5e	0b	db'.split("\t"),'e0	32	3a	0a	49	06	24	5c	c2	d3	ac	62	91	95	e4	79'.split("\t"),'e7	c8	37	6d	8d	d5	4e	a9	6c	56	f4	ea	65	7a	ae	08'.split("\t"),'ba	78	25	2e	1c	a6	b4	c6	e8	dd	74	1f	4b	bd	8b	8a'.split("\t"),'70	3e	b5	66	48	03	f6	0e	61	35	57	b9	86	c1	1d	9e'.split("\t"),'e1	f8	98	11	69	d9	8e	94	9b	1e	87	e9	ce	55	28	df'.split("\t"),'8c	a1	89	0d	bf	e6	42	68	41	99	2d	0f	b0	54	bb	16'.split("\t")]
s_box[3],s_box[5] = s_box[5], s_box[3]

## ByteSub Transformation(BS)

In [6]:
def ByteSub(input):
  substitue_list = [[] for i in range(4)]
  for i,val in enumerate(input):
    for j in val:
      substitue_list[i].append(s_box[int(j[0],16)][int(j[1],16)])
  return substitue_list

## ShiftRow Transformation(SR)

In [7]:
def ShiftRow(BS):
  for i in range(4):
    BS[i] = BS[i][i:]+BS[i][:i]
  return BS

## Mix Column(MC)

In [8]:
def galoisMultiplication(b, a):
    if b == 1:
        return a
    tmp = (a << 1) & 255
    if b == 2:
        return tmp if a < 128 else tmp ^ 27
    if b == 3:
        return galoisMultiplication(2, a) ^ a

    
def MixColumns(SR):
  M = [['02','03','01','01'],['01','02','03','01'],['01','01','02','03'],['03','01','01','02']]
  MC = [[] for i in range(4)]
  for i in range(4):    
    for j in range(4):  
      temp_val = 0x00
      for k in range(4):
        temp_val = temp_val ^ galoisMultiplication(int(M[i][k],16),int(SR[k][j],16))
      MC[i].append(format(temp_val,'#04x')[2:])
  return MC

## Key Schedule(KS)

In [9]:
def keygenerator(old_key,index):
  def T_of_W_KS(val,idx): 
    val = val[1:] + val[:1]
    ri = ['00','01','02','04','08','10','20','40','80','1b','36']
    sub_list = []
    for i in val:
      sub_list.append(s_box[int(i[0],16)][int(i[1],16)])
    temp = int(sub_list[0],16) ^ int(ri[idx],16)
    sub_list[0] = format(temp,'#04x')[2:]
    return sub_list
  def Xor_keys(list1,list2): 
    res = []
    for key_idx in range(4):
      res.append(format(int(list1[key_idx],16) ^ int(list2[key_idx],16),'#04x')[2:])
    return res

  new_key = []
  new_key.append(Xor_keys(old_key[0],T_of_W_KS(old_key[3],index)))
  new_key.append(Xor_keys(old_key[1],new_key[0]))
  new_key.append(Xor_keys(old_key[2],new_key[1]))
  new_key.append(Xor_keys(old_key[3],new_key[2]))

  return new_key

# Main Logic

In [10]:
Number_of_rounds = 11
result = []
mess="00 00 00 00 00 00 00 00 00 00 00 00 00 00 ab d0".split(" ")
key="1a 0c 24 f2 87 54 95 bc b7 08 0e 43 92 0f 56 80".split(" ")
mess = stringToList(mess)
key = stringToList(key)
for i in range(Number_of_rounds):
  if i == 0:
    print("_____________________________Round ",i,"_____________________________\n Input Msg:  ",printMessage(mess),"\n       Key:  ",printMessage(key))
    mess = TransformData(mess)
    key = TransformData(key)
    mess = RoundKeyAddition(mess,key)
    mess = TransformData(mess)
    key = TransformData(key)
  elif i == 10:
    key = keygenerator(key[:],i)
    print("_____________________________Round ",i,"_____________________________\n       Key:  ",printMessage(key))
    mess = TransformData(mess)
    key = TransformData(key)
    mess = RoundKeyAddition(ShiftRow(ByteSub(mess)),key)
    mess = TransformData(mess)
    key = TransformData(key)
  else:
    key = keygenerator(key[:],i)
    print("_____________________________Round ",i,"_____________________________\n       Key:  ",printMessage(key))
    mess = TransformData(mess)
    key = TransformData(key)
    mess = RoundKeyAddition(MixColumns(ShiftRow(ByteSub(mess))),key)
    mess = TransformData(mess)
    key = TransformData(key)
  print("\nOutput Msg:  ",printMessage(mess))

_____________________________Round  0 _____________________________
 Input Msg:   00 00 00 00 00 00 00 00 00 00 00 00 00 00 ab d0  
       Key:   1a 0c 24 f2 87 54 95 bc b7 08 0e 43 92 0f 56 80 

Output Msg:   1a 0c 24 f2 87 54 95 bc b7 08 0e 43 92 0f fd 50 
_____________________________Round  1 _____________________________
       Key:   6d 09 e9 bd ea 5d 7c 01 5d 55 72 42 cf 5a 24 c2 

Output Msg:   b5 79 12 fb 49 5f 73 55 dd 2f 6e 28 78 96 ef f3 
_____________________________Round  2 _____________________________
       Key:   ef 3f cc 37 05 62 b0 36 58 37 c2 74 97 6d e6 b6 

Output Msg:   53 b7 5e a3 9c 06 2a af 35 1b bb 42 8e 18 b5 38 
_____________________________Round  3 _____________________________
       Key:   d7 b1 82 bf d2 d3 32 89 8a e4 f0 fd 1d 89 16 4b 

Output Msg:   7b e3 5f b0 40 26 ec 9e 4e ed 55 0f 06 c1 03 74 
_____________________________Round  4 _____________________________
       Key:   78 f6 31 1b aa 25 03 92 20 c1 f3 6f 3d 48 e5 24 

Output Msg:   3c 11 7d 2